<a href="https://colab.research.google.com/github/pckuo/dale-regression/blob/main/playground_pck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qpsolvers[open_source_solvers]

In [2]:
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
from qpsolvers import solve_ls
import time
import pickle
# import jax.numpy as jnp
# from jax import grad, random, lax

### Train a Linear Perceptron with Quadratic Programming

In [3]:
SEED = 42

# simulation
# N = 10000         # n_neuorns
# alpha = 1        # proportional limit
noise_coeff = 0  # noise_level
n_runs = 1   # n_runs per N

alphas = np.array([1])
# Ns = np.logspace(2,4, num=4)
Ns = np.array([1e4])

In [4]:
# set seed
np.random.seed(SEED)

results_dict = {}

for alpha in alphas:
  for N in Ns:
    print(f'alpha {alpha}, N {N}')
    P = int(alpha * N)

    # save results
    results_dict_alpha_N = {
        'w_student': [],
        'mean_error_w': [],
        'mse_w': [],
        'mean_w_teacher_sq': [],
        'mean_w_student_sq': [],
        'mean_scalar_product_w_teacher_student': []
    }

    start = time.time()
    for run_id in range(n_runs):
      # teacher weights
      w_teacher = np.random.normal(size=int(N))

      # data
      X = np.random.normal(size=(P, int(N)))
      eps = noise_coeff * np.random.normal(size=P)
      y = X @ w_teacher + eps

      # solve with sklearn linear regression
      # reg_nnls = LinearRegression(positive=True)
      # w_student_sk = reg_nnls.fit(X, y).coef_

      # solve with quadratic programming
      w_student = solve_ls(
        R=X,
        s=y,
        G=-1*np.eye(int(N)),
        h=np.zeros(int(N)),
        solver="osqp"
      )
      # solver: osqp: 6min/ 9G for N=1e4
      # solver: quadprog: >15min/ 6G for N=1e4
      # solver: cvxopt: >11min/ 6G for N=1e4

      # compute errors
      # mean_error_w_sk = np.sum((w_student_sk - w_teacher))/ N
      mean_error_w = np.sum((w_student - w_teacher))/ N
      mse_w = np.sum((w_student - w_teacher)**2)/ N
      mean_w_teacher_sq = np.sum(w_teacher**2)/ N
      mean_w_student_sq = np.sum(w_student**2)/ N
      mean_scalar_product_w_teacher_student = np.sum(w_student * w_teacher)/ N

      # save results
      results_dict_alpha_N['w_student'].append(w_student)
      results_dict_alpha_N['mean_error_w'].append(mean_error_w)
      results_dict_alpha_N['mse_w'].append(mse_w)
      results_dict_alpha_N['mean_w_teacher_sq'].append(mean_w_teacher_sq)
      results_dict_alpha_N['mean_w_student_sq'].append(mean_w_student_sq)
      results_dict_alpha_N['mean_scalar_product_w_teacher_student'].append(mean_scalar_product_w_teacher_student)

    end = time.time()
    print(f' elapsed time: {end - start}')
    print(f' mse_w: {np.mean(results_dict_alpha_N["mse_w"])}')

    for k, v in results_dict_alpha_N.items():
      results_dict_alpha_N[k] = np.array(v)

    results_dict[(alpha, int(N))] = results_dict_alpha_N


alpha 1, N 10000.0


/usr/local/lib/python3.10/dist-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/qpsolvers/conversions/ensure_sparse_matrices.py:24: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(


 elapsed time: 382.1630709171295
 mse_w: 1.0168924646503388


In [ ]:
metric = 'mse_w'

colors = ['b', 'r']
fig, ax = plt.subplots(1,1, figsize=(6,4), dpi=200)
for alpha_id, alpha in enumerate(alphas):
  for N in Ns:
    ax.scatter(
        [N]*n_runs,
        results_dict[(alpha, int(N))][metric],
        c=colors[alpha_id]
    )
ax.set_xscale('log')
ax.set_xlabel('N')
ax.set_ylabel('generalization error')
ax.set_title('blue: alpha=0.75, red: alpha=1.25')



In [10]:
# MSE
print(f'mean_error_w: {mean_error_w}')
print(f'mse_w: {mse_w}')
# print(f'mean_w_teacher_sq: {mean_w_teacher_sq}')
# print(f'mean_w_student_sq: {mean_w_student_sq}')
# print(f'mean_scalar_product_w_teacher_student: {mean_scalar_product_w_teacher_student}')

mean_error_w: 0.5818401061397249
mse_w: 1.0168924646503388


In [ ]:
plt.plot(w_student - w_teacher)

In [ ]:
alpha = 1.25
for N in Ns:
  plt.hist(
      results_dict[(alpha, int(N))]['w_student'][0],
      bins=np.linspace(-2,5,20),
      density=True,
      histtype='step',
      label=f'{N}'
  )
plt.legend()

In [31]:
with open('./result_dict_preliminary.pkl', 'wb') as f:
  pickle.dump(results_dict, f)

In [43]:
from google.colab import files
files.download( "./result_dict_preliminary.pkl" )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>